In [321]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import ast
import os


## Flow

Load the model scenario data from REPORT_00 which contains at least 1 scenario as identified in the Scenario column
create a mapping of template workbook to scenario ID based on scenarios in REPORT

adjust the emissions factors for GWP to CO2eq in the model scenario data

Load the instructions called 'filters' from the folder and group them into one instruction list. The instructions are the same for every scenario. 
Group these instructions into one filter list

For each scenario:
    get workbook ID associated with this scenario and add it to the filter instruction 
    append to full_filter instruction set

extract data from REPORT 00 using the full_filter instruction set. 

populate each workbook in the mapping (above) with the extracted information

save workbook




### Define unit conversion factors, emissions factors

In [322]:
# Conversion of units in model to units used in reporting workbook

conversion_factors = {
    'Mt': 0.001, #units in model for emissions are already kt.
    'USD': 1/12.94, # to 2015 USD
    'kt': 1, #all emissions are in kt
    'GW': 1, 
    'TWh': 1/3.6, #PJ to TWh
    'EJ': 0.001, #PJ to EJ
    '*': 1,
    '': 1,
    'million inhab': 0.001,
    'gas_Nm3': 25641025,#Nm3 of NGas per PJ. convert PJ of NGas to nM3 of gas    
    'crude_Mt':1/42, #Mt of Ngas per PJ. convert PJ of crude oil to Mt of crude oil
    'Mtoe': 23884.6, #1 PJ = 23884.6 Mtoe, convert PJ of crude oil/biomass to Mtoe.
    'biomass_Mt':1/16, #Mt of biomass per PJ. based on 16MJ/kg convert PJ of biomass to Mt of biomass
    'H2_Nm':78740157, #Nm3 per PJ. convert PJ of hydrogen to Nm3
    'Mio' : 0.001,
    'pkm' : 0.000001, #bpkm default unit in the model
    'tkm' : 0.000001, #btkm default unit in the model
    'Gvkm' : 1, #bvkm default unit in the model
    
}



# set the emissions factors
#'Indicator' entries here must match the same spelling as in the REPORT
emf_df = pd.DataFrame({'Indicator': ['CH4','N2O','C2F6','CF4'],
                    'E_factor': [28, 265,11100,6630]})




## Define functions


In [323]:
# Filtering function for report_df
def correct_list_string(s):
    # Check if the string starts with [ and ends with ] to ensure it looks like a list
    s = s.replace('?', '')

    if s.startswith('[') and s.endswith(']'):
        # Extract the contents inside the brackets and split them by comma
        items = s[1:-1].split(',')
        
        # Process each item to ensure it has single quotes around it
        corrected_items = []
        for item in items:
            # Strip spaces and any extra single quotes
            item_stripped = item.strip().strip("'")
            corrected_items.append(f"'{item_stripped}'")
        
        # Reconstruct the list string
        return '[' + ', '.join(corrected_items) + ']'
    return s

def filter_data(row):
    conditions = []
    debug_info = [] # To collect debug information

    # Create the unique ID from the filter_df's first three columns
    unique_id = f"{row['WorkbookID']}_{row['SheetID']}_{row['RowID']}"

    for col, value in row.items():
        # Skip the first three columns, columns that are not in report_df, columns with '*' as value, 
        # and the 'Year' column
        if col in ['WorkbookID', 'SheetID', 'RowID'] or col not in report_df.columns or value == "*":
            continue

         # If value is a string, remove spaces at the end
        if isinstance(value, str):
            value = value.rstrip()

        # Check and correct the string representation of list if necessary
        if isinstance(value, str) and value.startswith("["):
            if col != 'Year':  # Only correct if the column is not 'Year'
                value = correct_list_string(value)  # Correct the string representation
            value = ast.literal_eval(value)  # Convert the string to an actual list

        # If value is a list, check if the column value in report_df is in the list
        if isinstance(value, list):
            conditions.append(report_df[col].isin(value))
            debug_info.append(f"{col} is in {value}") # For debugging
        else:
            conditions.append(report_df[col] == value)
            debug_info.append(f"{col} == {value}") # For debugging
            

    
    # If there are no conditions to check, return the entire report_df
    if not conditions:
        return report_df
    
    # Combine all conditions with an AND operator
    combined_condition = conditions[0]
    for cond in conditions[1:]:
        combined_condition &= cond

    result = report_df[combined_condition].copy()  # Ensure not to modify the original dataframe
    result['UniqueID'] = unique_id  # Add the unique ID to the filtered result
    if result.empty:
        print(f"Filter {debug_info} resulted in empty dataframe.") # This will show the failed conditions
        

    return result

# Convert SATIMGE values based on Unit column
def convert_value(row):
    factor = conversion_factors.get(row['TargetUnit'])
    if factor:
        result_value = row['SATIMGE'] * factor
        return np.round(result_value, 6)
    # If unit not found in conversion_factors, return original value
    return row['SATIMGE']


## Path names, and LOAD data

In [324]:
path_Report = "input/REPORT_00.csv" # add/replace file path here or filename if in same directory
path_SetsAndMaps = "input/SetsAndMaps.xlsm" # add/replace file path here or filename if in same directory
path_Output = "input/merged_data.csv" # add/replace file path here or filename if in same directory

# Read in the DataFrames
report_df = pd.read_csv(path_Report, low_memory=False)
mapPRC_df = pd.read_excel(path_SetsAndMaps, sheet_name="mapPRC")
mapCOM_df = pd.read_excel(path_SetsAndMaps, sheet_name="mapCOM")

# Replace 'Eps' values with 0
report_df['SATIMGE'] = report_df['SATIMGE'].replace('Eps', 0)
report_df['SATIMGE'] = report_df['SATIMGE'].astype(float)

# Merge the DataFrames
merged_df = report_df.merge(mapPRC_df, on='Process', how='left')
merged_df = merged_df.merge(mapCOM_df, on='Commodity', how='left')

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# merged_df.to_csv(path_Output, index=False)


In [325]:
report_df.head(10)

,Process,Commodity,Year,Scenario,Indicator,SATIMGE
0,SATIMMarginals,CO2CAPT,2040,netzero_9_imagine,Marginals,-0.129881
1,SATIMMarginals,CO2CAPT,2041,netzero_9_imagine,Marginals,-0.348815
2,SATIMMarginals,CO2CAPT,2042,netzero_9_imagine,Marginals,-0.438883
3,SATIMMarginals,CO2CAPT,2043,netzero_9_imagine,Marginals,-0.438883
4,SATIMMarginals,CO2CAPT,2044,netzero_9_imagine,Marginals,-0.438883
5,SATIMMarginals,CO2CAPT,2045,netzero_9_imagine,Marginals,-0.438883
6,SATIMMarginals,CO2CAPT,2046,netzero_9_imagine,Marginals,-0.438883
7,SATIMMarginals,CO2CAPT,2047,netzero_9_imagine,Marginals,-0.438883
8,SATIMMarginals,CO2CAPT,2048,netzero_9_imagine,Marginals,-0.438883
9,SATIMMarginals,CO2CAPT,2049,netzero_9_imagine,Marginals,-0.438883


## Create workbook and scenario mapping

In [326]:
# Extract unique scenarios
unique_scenarios = report_df['Scenario'].unique()

# Create a mapping of WorkbookID values to corresponding scenario names

workbook_scenario_mapping = {i: scenario for i, scenario in enumerate(unique_scenarios,start =1 )}

workbook_scenario_mapping

{1: 'netzero_9_imagine'}

In [327]:
# create wb_filenames list for iterating over the workbooks to populate

#  Determine the number of unique scenarios
num_scenarios = len(unique_scenarios)

# Create the list of file names
wb_filenames = [f'input/WB{i}.xlsx' for i in range(1, num_scenarios + 1)]

wb_filenames

['input/WB1.xlsx']

### Adjust Emissions factors to CO2eq

In [328]:
#ADJUST THE EMISSIONS TO CO2EQ:


# Merge the two DataFrames based on the 'Indicator' column
merged_df = merged_df.merge(emf_df, on='Indicator', how='left')

# Fill missing values in 'Multiplier' column with 1
merged_df['E_factor'].fillna(1, inplace=True)

# Multiply the values from 'SATIMGE' by the 'E_factor' column
merged_df['Result'] = merged_df['SATIMGE'] * merged_df['E_factor']

# Optionally, drop the 'Multiplier' column or rename columns
merged_df = merged_df.drop('SATIMGE', axis=1) #dropping the original SATIMGE values.
merged_df = merged_df.drop('E_factor', axis=1)
#rename to new SATIMGE
merged_df = merged_df.rename(columns={'Result': 'SATIMGE'}) #new SATIMGE values

# Reset index
merged_df.reset_index(drop=True, inplace=True)

merged_df.head(15)

,Process,Commodity,Year,Scenario,Indicator,Sector,Subsector,Subsubsector,TechDescription,IPCC_Category_L1,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,SATIMGE
0,SATIMMarginals,CO2CAPT,2040,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.129881
1,SATIMMarginals,CO2CAPT,2041,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.348815
2,SATIMMarginals,CO2CAPT,2042,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883
3,SATIMMarginals,CO2CAPT,2043,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883
4,SATIMMarginals,CO2CAPT,2044,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883
5,SATIMMarginals,CO2CAPT,2045,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883
6,SATIMMarginals,CO2CAPT,2046,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883
7,SATIMMarginals,CO2CAPT,2047,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883
8,SATIMMarginals,CO2CAPT,2048,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883
9,SATIMMarginals,CO2CAPT,2049,netzero_9_imagine,Marginals,All,All,All,SATIMMarginals Annual,NaN,NaN,NaN,NaN,CO2 Captured ...,CO2 Captured ...,0,-0.438883


In [329]:
scenarios = merged_df['Scenario'].unique() #array 
scenarios

array(['netzero_9_imagine'], dtype=object)

## Load all filter instructions into one df

In [330]:

#path where all filters are
filter_folder = 'input\\Filters\\'
curr_wdr = os.getcwd()
filters_path = os.path.join(curr_wdr,filter_folder)

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate over files in the directory
for filename in os.listdir(filters_path):
    if filename.endswith('.xlsx'):
        print(filename)
        file_path = os.path.join(filters_path, filename)
        df = pd.read_excel(file_path)
        df = df.dropna()
        dataframes.append(df)

# Concatenate all DataFrames into one
all_filters_df = pd.concat(dataframes, ignore_index=True)

# Drop rows with NaN values
all_filters_df = all_filters_df.dropna()

# Fill NaN values with 0 for the specified columns
columns_to_convert = ['SheetID', 'RowID']

if not all_filters_df.empty:
    all_filters_df[columns_to_convert] = all_filters_df[columns_to_convert].fillna(0).astype(int)


commerce_filters.xlsx
EXTRACTIVE ENERGY INDUSTR (DB) filters.xlsx
freight_transport_filter.xlsx
j_filter_AFOLU.xlsx
j_filter_Industry_EII.xlsx
j_filter_Industry_Light.xlsx
j_filter_Waste.xlsx
new_other_energy_industries.xlsx
pass_transport_filter.xlsx
Power_DB filter.xlsx
res_filters.xlsx


### rename the scenarios according to Workbook ID

In [331]:
# # Define a function to update the 'Scenario' column based on conditions
# def update_scenario(row):
#     scenario = row['Scenario']
#     workbook_id = row['WorkbookID']
    
#     # Check if the scenario name is in the mapping and matches the WorkbookID
#     if scenario in workbook_scenario_mapping and workbook_scenario_mapping[workbook_id] == scenario:
#         return scenario  # No change is needed
        
#     # Check if the scenario name is in the mapping but doesn't match the WorkbookID
#     if scenario in workbook_scenario_mapping and workbook_scenario_mapping[workbook_id] != scenario:
#         return workbook_scenario_mapping[workbook_id]  # Update to the correct scenario
    
#     # If the scenario is not in the mapping, return the scenario based on WorkbookID
#     if workbook_id in workbook_scenario_mapping:
#         return workbook_scenario_mapping[workbook_id]
    
#     # If neither scenario nor WorkbookID are in the mapping, return the existing scenario
#     return scenario

# # Update the 'Scenario' column in filter_df based on the defined conditions
# filter_df['Scenario'] = filter_df.apply(update_scenario, axis=1)




In [332]:
all_filters_df.head(5)

,SheetID,RowID,Process,Commodity,Year,Indicator,SATIMGE,Sector,Subsector,Subsubsector,TechDescription,IPCC_Category_L1,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,TargetUnit
0,18,38,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,*,*,*,*,*,*,*,*,*,EJ
1,18,40,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Space Heating'],*,*,*,*,*,*,*,*,EJ
2,18,41,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Cooling'],*,*,*,*,*,*,*,*,EJ
3,18,42,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Cooking'],*,*,*,*,*,*,*,*,EJ
4,18,43,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Water Heating'],*,*,*,*,*,*,*,*,EJ


### Apply WorkbookID corresponding to the workbook mapping

take the filter_df and for each workbook ID create a coopy of filter_df add WorkbookID column corresponding to that scenario, and then repeat and append for each scenario

In [333]:
#apply it to filter_df for each scenario and corresponding workbook ID

In [334]:

#for each scenario in the workbook mapping (which contains a list of unique scenarios from the report file)
    #get the ID from the map
    #add workbookID column to a copy of base_filters and add the Workbook ID from the map
    #append this copy to full_filter_df which is now the complete filter
    

full_filter_df = pd.DataFrame()

for wb_id, value in workbook_scenario_mapping.items():
    print(f"ID: {id}, scenario: {value}")
    temp = all_filters_df
    
    #add ID
    temp['WorkbookID'] = wb_id
    
    #append
    full_filter_df = pd.concat([full_filter_df, temp], ignore_index=True)
    

full_filter_df.head(5)

ID: 1, scenario: netzero_9_imagine


,SheetID,RowID,Process,Commodity,Year,Indicator,SATIMGE,Sector,Subsector,Subsubsector,TechDescription,IPCC_Category_L1,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,TargetUnit,WorkbookID
0,18,38,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,*,*,*,*,*,*,*,*,*,EJ,1
1,18,40,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Space Heating'],*,*,*,*,*,*,*,*,EJ,1
2,18,41,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Cooling'],*,*,*,*,*,*,*,*,EJ,1
3,18,42,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Cooking'],*,*,*,*,*,*,*,*,EJ,1
4,18,43,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Water Heating'],*,*,*,*,*,*,*,*,EJ,1


In [335]:
# Split the 'TargetUnit' column if needed
Units = full_filter_df[['WorkbookID','SheetID', 'RowID', 'TargetUnit']]
all_filters_df = all_filters_df.drop(columns=['TargetUnit'])

# Reset the index
all_filters_df.reset_index(drop=True, inplace=True)


# Now, 'all_filters_df' contains data from all the Excel files in the folder
filter_df = all_filters_df

In [336]:
filter_df.head(15)

,SheetID,RowID,Process,Commodity,Year,Indicator,SATIMGE,Sector,Subsector,Subsubsector,TechDescription,IPCC_Category_L1,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,WorkbookID
0,18,38,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,*,*,*,*,*,*,*,*,*,1
1,18,40,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Space Heating'],*,*,*,*,*,*,*,*,1
2,18,41,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Cooling'],*,*,*,*,*,*,*,*,1
3,18,42,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Cooking'],*,*,*,*,*,*,*,*,1
4,18,43,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Water Heating'],*,*,*,*,*,*,*,*,1
5,18,44,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Lighting'],*,*,*,*,*,*,*,*,1
6,18,45,*,*,"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Other'],*,*,*,*,*,*,*,*,1
7,18,47,*,"['COMOLP', 'COMOKE', 'COMOHF', 'COMODS', 'COMO...","[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Space Heating'],*,*,*,*,*,*,*,*,1
8,18,48,*,['COMGAS'],"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Space Heating'],*,*,*,*,*,*,*,*,1
9,18,49,*,['COMCOA'],"[2019, 2030, 2040, 2050]",['FlowIn'],*,['Commerce'],*,['Space Heating'],*,*,*,*,*,*,*,*,1


In [337]:
Units['UniqueID'] = Units['WorkbookID'].astype(str) + '_' + Units['SheetID'].astype(str) + '_' + Units['RowID'].astype(str)
Units.drop(columns=['WorkbookID', 'SheetID', 'RowID'], inplace=True)

Units['UniqueID']

C:\Users\bryce\AppData\Local\Temp\ipykernel_14056\1063592531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Units['UniqueID'] = Units['WorkbookID'].astype(str) + '_' + Units['SheetID'].astype(str) + '_' + Units['RowID'].astype(str)
C:\Users\bryce\AppData\Local\Temp\ipykernel_14056\1063592531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Units.drop(columns=['WorkbookID', 'SheetID', 'RowID'], inplace=True)


0        1_18_38
1        1_18_40
2        1_18_41
3        1_18_42
4        1_18_43
          ...   
1233    1_16_123
1234    1_16_124
1235    1_16_126
1236    1_16_130
1237    1_16_133
Name: UniqueID, Length: 1238, dtype: object

In [338]:
columns = ['Process', 'Commodity', 'Year', 'Scenario', 'Indicator',
       'Sector', 'Subsector', 'Subsubsector', 'TechDescription',
       'IPCC_Category_L1', 'IPCC_Category_L2', 'IPCC_Category_L3',
       'IPCC_Category_L4', 'Description', 'Short Description',
       'Commodity_Name']

#for col in columns:
 #   print(col, " unique values: \n", merged_df[col].unique(), "\n \n", '--------------------------------########################--------------------', '\n \n')


In [339]:
report_df = merged_df

# Instantiate df list
filtered_dfs = []
i = 1
# Iterate through each row in filter_df and filter report_df
for _, row in filter_df.iterrows():
    filtered_dfs.append(filter_data(row))
    

# Concatenate all dataframes in filtered_dfs to get a single dataframe
final_df = pd.concat(filtered_dfs, ignore_index=True)

# Export 
# final_df.to_csv('input/results_filter.csv', index = False)

final_df.reset_index(drop=True, inplace=True)


final_df


Filter ["Commodity is in ['COMOLP', 'COMOKE', 'COMOHF', 'COMODS', 'COMOGS']", 'Year is in [2019, 2030, 2040, 2050]', "Indicator is in ['FlowIn']", "Sector is in ['Commerce']", "Subsubsector is in ['Space Heating']"] resulted in empty dataframe.
Filter ["Commodity is in ['COMOLP', 'COMOKE', 'COMOHF', 'COMODS', 'COMOGS']", 'Year is in [2019, 2030, 2040, 2050]', "Indicator is in ['FlowIn']", "Sector is in ['Commerce']", "Subsubsector is in ['Cooling']"] resulted in empty dataframe.
Filter ["Commodity is in ['COMGAS']", 'Year is in [2019, 2030, 2040, 2050]', "Indicator is in ['FlowIn']", "Sector is in ['Commerce']", "Subsubsector is in ['Cooling']"] resulted in empty dataframe.
Filter ["Commodity is in ['COMCOA']", 'Year is in [2019, 2030, 2040, 2050]', "Indicator is in ['FlowIn']", "Sector is in ['Commerce']", "Subsubsector is in ['Cooling']"] resulted in empty dataframe.
Filter ["Commodity is in ['COMGAS']", 'Year is in [2019, 2030, 2040, 2050]', "Indicator is in ['FlowIn']", "Sector is 

,Process,Commodity,Year,Scenario,Indicator,Sector,Subsector,Subsubsector,TechDescription,IPCC_Category_L1,IPCC_Category_L2,IPCC_Category_L3,IPCC_Category_L4,Description,Short Description,Commodity_Name,SATIMGE,UniqueID
0,CECELCCEN-E,COMELC,2019,netzero_9_imagine,FlowIn,Commerce,ComExisting,Cooling,Commercial - Cooling - Electricity - Central -...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4a Commercial/Institutional,Commercial - Electricity ...,Electricity,0,27.275927,1_18_38
1,CECELCCEN-E,COMELC,2030,netzero_9_imagine,FlowIn,Commerce,ComExisting,Cooling,Commercial - Cooling - Electricity - Central -...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4a Commercial/Institutional,Commercial - Electricity ...,Electricity,0,26.792673,1_18_38
2,CECELCCEN-E,COMELC,2040,netzero_9_imagine,FlowIn,Commerce,ComExisting,Cooling,Commercial - Cooling - Electricity - Central -...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4a Commercial/Institutional,Commercial - Electricity ...,Electricity,0,12.075822,1_18_38
3,CECELCCEN-N,COMELC,2030,netzero_9_imagine,FlowIn,Commerce,ComExisting,Cooling,Commercial - Cooling - Electricity - Central -...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4a Commercial/Institutional,Commercial - Electricity ...,Electricity,0,1.918835,1_18_38
4,CECELCCEN-N,COMELC,2040,netzero_9_imagine,FlowIn,Commerce,ComExisting,Cooling,Commercial - Cooling - Electricity - Central -...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4a Commercial/Institutional,Commercial - Electricity ...,Electricity,0,19.870677,1_18_38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17956,RMEWOLPKETLPG-E,RESOLP,2030,netzero_9_imagine,N2O,Residential,MiddleIncome,Water Heating,Residential - Water Heating - Middle Income El...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4b Residential,Residential - Oil LPG ...,LPG,LPG,0.073736,1_16_133
17957,RMEWOLPKETLPG-E,RESOLP,2040,netzero_9_imagine,CH4,Residential,MiddleIncome,Water Heating,Residential - Water Heating - Middle Income El...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4b Residential,Residential - Oil LPG ...,LPG,LPG,0.232925,1_16_133
17958,RMEWOLPKETLPG-E,RESOLP,2040,netzero_9_imagine,N2O,Residential,MiddleIncome,Water Heating,Residential - Water Heating - Middle Income El...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4b Residential,Residential - Oil LPG ...,LPG,LPG,0.073482,1_16_133
17959,RMEWOLPKETLPG-E,RESOLP,2050,netzero_9_imagine,CH4,Residential,MiddleIncome,Water Heating,Residential - Water Heating - Middle Income El...,1 Energy,1A Fuel Combustion Activities,1A4 Other Sectors,1A4b Residential,Residential - Oil LPG ...,LPG,LPG,0.310297,1_16_133


In [340]:
# Initialize an empty list
cumulativeIDs = []

# Loop through N values
for N in [1, 2, 3]:
    # Loop through M values
    for M in range(109, 124):  # This will loop from 127 to 143 inclusive
        # Construct the string using the pattern and append to the list
        cumulativeIDs.append(f"{N}_24_{M}") # 24 = power tab. 
        
# Print the list
print(cumulativeIDs)


['1_24_109', '1_24_110', '1_24_111', '1_24_112', '1_24_113', '1_24_114', '1_24_115', '1_24_116', '1_24_117', '1_24_118', '1_24_119', '1_24_120', '1_24_121', '1_24_122', '1_24_123', '2_24_109', '2_24_110', '2_24_111', '2_24_112', '2_24_113', '2_24_114', '2_24_115', '2_24_116', '2_24_117', '2_24_118', '2_24_119', '2_24_120', '2_24_121', '2_24_122', '2_24_123', '3_24_109', '3_24_110', '3_24_111', '3_24_112', '3_24_113', '3_24_114', '3_24_115', '3_24_116', '3_24_117', '3_24_118', '3_24_119', '3_24_120', '3_24_121', '3_24_122', '3_24_123']


In [341]:
# Predefined year_to_column mapping.
year_to_column = {2020: 4, 2030: 5, 2040: 6, 2050: 7, 2060: 8}

# Aggregate the data by UniqueID and Year, summing SATIMGE values.
aggregated = final_df.groupby(['UniqueID', 'Year'])['SATIMGE'].sum().reset_index()

# Define the range of 'certain years'.
start_year = 2020  # adjust accordingly
end_year = 2060    # adjust accordingly

# Filter out the special rows using the cumulativeIDs list.
special_rows = aggregated[aggregated['UniqueID'].isin(cumulativeIDs)]

# For these special rows, compute cumulative sums in ten-year steps.
cumulative_sums = []

for unique_id in cumulativeIDs:
    subset = special_rows[special_rows['UniqueID'] == unique_id]
    for year in range(start_year, end_year, 10):
        end = year + 10
        total_sum = subset[(subset['Year'] >= year) & (subset['Year'] < end)]['SATIMGE'].sum()
        cumulative_sums.append((unique_id, year, total_sum))

# Convert the result into a DataFrame.
cumulative_df = pd.DataFrame(cumulative_sums, columns=['UniqueID', 'StartYear', 'CumulativeSATIMGE'])


cumulative_df.head(20)

,UniqueID,StartYear,CumulativeSATIMGE
0,1_24_109,2020,0.0
1,1_24_109,2030,0.0
2,1_24_109,2040,0.0
3,1_24_109,2050,0.0
4,1_24_110,2020,0.0
5,1_24_110,2030,0.0
6,1_24_110,2040,0.0
7,1_24_110,2050,0.0
8,1_24_111,2020,0.0
9,1_24_111,2030,0.0


## Make unit conversions

In [342]:
aggregated = pd.merge(aggregated, Units, on='UniqueID', how='left')

aggregated['SATIMGE'] = aggregated.apply(convert_value, axis=1)

# Populate the workbooks with data

### Create a WB for each scenario
from the base IDDRI workbook template

In [343]:
# The original blank template file
original_file = 'DDPReportingTemplate_vDec2023.xlsx'
template_folder = 'backup\\'
curr_wdr = os.getcwd()
template_file_path = os.path.join(curr_wdr,template_folder,original_file)


# Loop through the scenario_dict
for id, scenario in workbook_scenario_mapping.items():
    # Load the original workbook
    wb = load_workbook(template_file_path)
    
    # Get the specific sheet you want to modify
    sheet = wb['User guide']
    
    # Write the scenario to cell B14
    sheet['B12'] = scenario
    
    # Create the new filename
    new_filename = f'WB{id}.xlsx'
    
    # Save the new workbook with the updated name
    wb.save(os.path.join(curr_wdr,"input\\",new_filename))
    
    print(f"Created {new_filename} with scenario '{scenario}' in cell B12 of 'User guide' sheet")

Created WB1.xlsx with scenario 'netzero_9_imagine' in cell B12 of 'User guide' sheet


In [344]:
decades = list(range(start_year, end_year, 10))  # Adjust as per your requirement
cumulative_sums = {}

for unique_id, group in aggregated.groupby('UniqueID'):
    if unique_id in cumulativeIDs:
        decade_sums = {}
        
        for start_year in decades:
            end_year = start_year + 10
            decade_data = group[(group['Year'] >= start_year) & (group['Year'] < end_year)]
            decade_sums[start_year] = decade_data['SATIMGE'].sum()
        
        cumulative_sums[unique_id] = decade_sums

# Print out some of the cumulative_sums for debugging
for key, value in list(cumulative_sums.items())[:5]:
    print(key, value)


In [345]:
# 
aggregated = aggregated[~aggregated['UniqueID'].isin(cumulativeIDs)] #tilda is negation. aggregated NOT in...
aggregated = aggregated[aggregated['Year'].isin([2020,2030,2040,2050,2060,2070])]

In [346]:
decade_to_column = {
    2020: 4,
    2030: 5,
    2040: 6,
    2050: 7,
    2060: 8,
    2070: 9,
    # Adjust as needed
}

BASE_COLUMN = 4  # Adjust as needed

grouped = aggregated.groupby(['UniqueID'])

for (unique_id_string,), group in grouped:
    workbook_id, sheet_id, row_id = map(int, unique_id_string.split('_'))
    print(f"processing workbook {workbook_id}, sheet {sheet_id}, row {row_id}")

    # Use the workbook_id to select the correct workbook filename
    if 0 < workbook_id <= len(wb_filenames):
        file_name = wb_filenames[workbook_id - 1]  # Adjust for 0-based indexing
    else:
        raise ValueError(f"Workbook ID {workbook_id} is out of range.")
    
    try:
        book = load_workbook(file_name)
    except FileNotFoundError:
        print(f"File {file_name} not found. Skipping to the next file.")
        continue

    sheet_names = book.sheetnames
    if 0 <= sheet_id - 1 < len(sheet_names):
        sheet_name = sheet_names[sheet_id - 1]
        sheet = book[sheet_name]
    else:
        raise ValueError(f"Sheet ID {sheet_id} is out of range for workbook {file_name}.")

    # Check if the unique_id_string is in the special rows
    # Check if the unique_id_string is in the special rows and exists in cumulative_sums
    if unique_id_string in cumulativeIDs and unique_id_string in cumulative_sums:
            print(f'Entering cumulative logic for {unique_id_string}')
            for decade_start, sum_val in cumulative_sums[unique_id_string].items():
                column_index = decade_to_column[decade_start]
                sheet.cell(row=row_id, column=column_index, value=sum_val)
    else:
        # print(f'Entering non-cumulative logic for {unique_id_string}')
        for _, row in group.iterrows():
            if row['Year'] in year_to_column:
                column_index = year_to_column[row['Year']]
                sheet.cell(row=row_id, column=column_index, value=row['SATIMGE'])
            else:
                print(f"Warning: Year {row['Year']} not in year_to_column for unique_id_string {unique_id_string}")

    book.save(file_name)

print("Done")


processing workbook 1, sheet 10, row 100
processing workbook 1, sheet 10, row 102
processing workbook 1, sheet 10, row 104
processing workbook 1, sheet 10, row 106
processing workbook 1, sheet 10, row 108
processing workbook 1, sheet 10, row 110
processing workbook 1, sheet 10, row 112
processing workbook 1, sheet 10, row 114
processing workbook 1, sheet 10, row 118
processing workbook 1, sheet 10, row 120
processing workbook 1, sheet 10, row 141
processing workbook 1, sheet 10, row 143
processing workbook 1, sheet 10, row 144
processing workbook 1, sheet 10, row 149
processing workbook 1, sheet 10, row 153
processing workbook 1, sheet 10, row 155
processing workbook 1, sheet 10, row 158
processing workbook 1, sheet 10, row 160
processing workbook 1, sheet 10, row 163
processing workbook 1, sheet 10, row 164
processing workbook 1, sheet 10, row 166
processing workbook 1, sheet 10, row 169
processing workbook 1, sheet 10, row 171
processing workbook 1, sheet 10, row 173
processing workb